In [11]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from nni.algorithms.compression.pytorch.quantization import LsqQuantizer, QAT_Quantizer

import torch.nn as nn
import onnx
import onnx.numpy_helper
from math import ceil
### markus
import numpy as np
import PIL
import os
from re import L
import subprocess
from subprocess import DEVNULL, STDOUT
from xmlrpc.client import boolean
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
#%matplotlib notebook
import ProfileClass as prof


imgSizeW= 68#1280
imgSizeH= 18#720
gen_img_input_dim_w = imgSizeW
gen_img_input_dim_h = imgSizeH
gen_img_input_channels = 3

# class Mnist(nn.Module):
#     def __init__(self, input_channels, out_channels):
#         super(Mnist, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels = input_channels, out_channels = out_channels, kernel_size = 5, stride = 1)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         return x

# model = Mnist(gen_img_input_channels, 64)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# loop_run = '10'
# MP = prof.ModelProfile()
# [ProfileArray,status] = MP.doProfiling(model,
#                  loop_run,
#                  gen_img_input_channels,
#                  gen_img_input_dim_h,
#                  gen_img_input_dim_w,
#                  debug=True)

# accData=ProfileArray.values.tolist()

In [ ]:
imgSizeW= 68#1280

imgSizeH= 18#720
gen_img_input_dim_w = imgSizeW
gen_img_input_dim_h = imgSizeH
gen_img_input_channels = 3
#test input image
test_input_data="../convertdemo/dataset/rand_3.jpg"
#Paths
quant_image_path = "../quantization_images"
script_path = "../scripts"
log_path = "../logs"
network_path = "../convertdemo/network"
#Files
perform_script = "perform_r6.sh"
parse_script = "parse_r1.sh"
perform_log_file = "model_execution.log"
parsed_log_file = "model_execution_parsed.log"
model_name="mnist"


# Generate images based on some arbitrary input dimension

In [ ]:
path = quant_image_path

def generate_random_images(xdim, ydim, channels=3, count=1, path="."):
    """
    This functions generates random bmp images to use for quantization given
    a defined dimension
        @xdim   .. width of images
        @ydim   .. height of images
        @count  .. number of images
        @path   .. path of images
    """

    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path)
        print("The new directory quantization_images is created!")

    #delete the pre generated bmp/jpg files
    filelist = [ f for f in os.listdir(quant_image_path) if (f.endswith(".jpg") or f.endswith(".bmp") ) ]
    for f in filelist:
        os.remove(os.path.join(quant_image_path, f))
        
    for c in range(count):
        rnd_img = np.random.randint(low=0,high=255, size=(ydim, xdim, channels),dtype=np.uint8) #imag.transpose((1,2,0)
        imag_tp = np.ascontiguousarray(rnd_img, dtype=np.uint8)

        pil_image = PIL.Image.frombytes('RGB',(xdim, ydim), imag_tp)
        pil_image.save(path + "/rand_"+str(c)+".bmp")
        pil_image.save(path + "/rand_"+str(c)+".jpg")



In [ ]:

torch.manual_seed(0)
# choose the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## generate 10 random input images based on the provided dimensions
generate_random_images(gen_img_input_dim_w, gen_img_input_dim_h, gen_img_input_channels, count=20, path=quant_image_path)


# Build the normal model

In [29]:
import math
import torch
import torch.nn as nn
from builder import *
base_factor = 32
class slice_reshape_operation(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op(x)

        return x
        
        

In [35]:
class slice_reshape_operation_2(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation_2, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op1 = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        self.op2 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op3 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op1(x)
        x = self.op2(x)
        x = self.op3(x)
        return x

In [36]:
op_name = "ir_k3_re"

C_in = 4
C_out = 4
img_w = 10
img_h = 10
expansion = 6
stride = 1

#generate input to test
input_x = torch.rand(1,3,base_factor*base_factor ,img_w*img_h*C_in*base_factor)

print(input_x.shape)
model = slice_reshape_operation_2(op_name, C_in, C_out, img_w, img_h, expansion, stride)

y = model(input_x)

torch.Size([1, 3, 1024, 12800])


In [ ]:
class Mnist(nn.Module):
    def __init__(self, input_channels, out_channels):
        super(Mnist, self).__init__()
        #self.conv1 = nn.Conv2d(num_channels, 20, 5, 1)
        self.conv1 = nn.Conv2d(in_channels = input_channels, out_channels = out_channels, kernel_size = 5, stride = 1)
        # self.conv2 = nn.Conv2d(20, 20, 5, 1)

        # self.conv3 = nn.Conv2d(20, 50, 1, 1)

        # f1 = int(imgSizeX/4) - 3
        # f2 = int(imgSizeY/4) - 3
        # self.fc1 = nn.Linear(f1*f2*50, 500)
        # self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # x = F.max_pool2d(x, 2, 2)
        # x = F.relu(self.conv2(x))

        # x = F.relu(self.conv3(x))

        # x = F.max_pool2d(x, 2, 2)
        # f1 = int(imgSizeX/4) - 3
        # f2 = int(imgSizeY/4) - 3
        # x = x.view(-1, f1*f2*50)
        # x = F.relu(self.fc1(x))
        # x = self.fc2(x)
        return x

In [ ]:
class SimpleNN(nn.Module):

    def __init__(self):
        super(SimpleNN, self).__init__()
    
        self.fc1 = nn.Linear(gen_img_input_dim_w*gen_img_input_dim_h, 50)#gen_img_input_dim_w*gen_img_input_dim_h
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # select first dim from [1,C,H,W] ie. [0,:,:,:]
        x = x.select(0,0)
        #print(x.shape)
        # select first channel [0,:,:]
        x = x.select(0,0)
        #print(x.shape)
        x = x.reshape((1,-1))
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = self.fc2(x)
        #print(x.shape)
        #print(x)
        return F.log_softmax(x, dim=1)

In [ ]:
#model = SimpleNN()
model = Mnist(gen_img_input_channels, 64)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the model for random images

In [ ]:
import glob

def load_images(count=1, path=".", extension='*.bmp'):
    """
    loads images as np arrays; no normalization
    """
    imgs =  []
    files = glob.glob(path + "/" + extension)
    for i, f in enumerate(files):
        img = np.asarray(PIL.Image.open(f), dtype=np.float32).transpose((2,0,1))
        img = img.reshape( (1,img.shape[0],img.shape[1], img.shape[2]) )
        img = torch.from_numpy(img)
        imgs.append(img)

        if i+1 > count:
            break

    return imgs

imgs = load_images(count=2,path=quant_image_path)


In [ ]:
# model.train()
# for img in imgs:
#     target = torch.tensor([2])
#     optimizer.zero_grad()
#     output = model(img)
#     #print(output)
#     loss = F.nll_loss(output, target)
#     print(loss.item())
#     #print(target, output)
#     loss.backward()
#     optimizer.step()

In [37]:
#model = slice_reshape_operation(op_name, C_in, C_out, img_w, img_h, expansion, stride)
model = slice_reshape_operation_2(op_name, C_in, C_out, img_w, img_h, expansion, stride)

In [38]:
gen_img_input_channels = 3
gen_img_input_dim_h = base_factor*base_factor
gen_img_input_dim_w = img_w*img_h*C_in*base_factor

In [40]:
loop_run = '10'
MP = prof.ModelProfile()
[ProfileArray,status] = MP.doProfiling(model,
                 loop_run,
                 gen_img_input_channels,
                 gen_img_input_dim_h,
                 gen_img_input_dim_w,
                 debug=True)

accData=ProfileArray.values.tolist()

Creating ModelProfile Class...
------------- Exporting to onnx


Exported graph: graph(%input : Float(1, 3, 1024, 12800, strides=[39321600, 13107200, 12800, 1], requires_grad=0, device=cpu),
      %onnx::Conv_95 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_96 : Float(768, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_98 : Float(768, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_101 : Float(128, 768, 1, 1, strides=[768, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_102 : Float(128, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_104 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_107 : Float(768, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_110 : Float(128, 768, 1, 1, strides=[768, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_113 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_116 : Float

vnn_mnist.c: In function ‘vnn_CreateMnist’:
vnn_mnist.c:146:29: warning: unused variable ‘data’ [-Wunused-variable]
  146 |     uint8_t *               data;
      |                             ^~~~
At top level:
vnn_mnist.c:94:17: warning: ‘load_data’ defined but not used [-Wunused-function]
   94 | static uint8_t* load_data
      |                 ^~~~~~~~~


  COMPILE /home/khadas/nbg_unify_mnist/main.c
  COMPILE /home/khadas/nbg_unify_mnist/vnn_post_process.c
make: Nothing to be done for 'all'.
>>>>> Compiling the model on Remote Khadas Done! <<<<<<<
>>>>> Copying test input image to Remote... <<<<<<<
>>>>> Copying test input image to Remote! <<<<<<<
>>>>> Executing the model on Remote Khadas... <<<<<<<
>>>>> Executing the model on Remote Khadas Done! <<<<<<<
>>>>> Executing network on Khadas is saved in model_execution.log <<<<<<<
------------- Performing the profiling done!
------------- Parsing the profiling results...
Parsing ../logs/model_execution.log....
------------- Parsing the profiling results done!
------------- auto_profile done!...


In [15]:
#subprocess.check_call(['../scripts/sajjad.sh'])
#subprocess.check_call(["../scripts/sajjad.sh"])
#!ls -la
#!echo "Hello"
#!bash ../scripts/perform_r6.sh 10  ../convertdemo/dataset/rand_3.jpg ../logs/model_execution.log > ../logs/jupyter.log
#myscript="../scripts/sajjad.sh"
#!bash {myscript}
 

In [ ]:
Run#1 4 cin 4cout number 1

['1365544' '2695579' '272934' '271745' '271674' '271654' '272228' '271578'
 '271715' '274609' '272346' '271730' '2722444' '272244']


Run#2 4 cin 4cout number 2

['97151' '2356437' '351619' '353184' '351332' '351130' '351330' '354675'
 '351358' '351234' '351391' '351176' '3518616' '351861']
Run#3 4 cin 4cout number 3
['84047' '38567' '524027' '522490' '522432' '522506' '523456' '522375'
 '522196' '522294' '523748' '522288' '5228013' '522801']


In [41]:
C0 = np.array(ProfileArray[0])
C1 = np.array(ProfileArray[1])
C2 = np.array(ProfileArray[2])
print('---------\n')
print(C0)
print('---------\n')
print(C1)
print('---------\n')
print(C2)
print('---------\n')
print(status)



---------

['Create Neural Network' 'Verify Graph' 'Run the 1 time' 'Run the 2 time'
 'Run the 3 time' 'Run the 4 time' 'Run the 5 time' 'Run the 6 time'
 'Run the 7 time' 'Run the 8 time' 'Run the 9 time' 'Run the 10 time'
 'Total   ' 'Average ']
---------

[' 84047us' ' 38567us' ' 524027.00us' ' 522490.00us' ' 522432.00us'
 ' 522506.00us' ' 523456.00us' ' 522375.00us' ' 522196.00us'
 ' 522294.00us' ' 523748.00us' ' 522288.00us' ' 5228013.00us'
 ' 522801.31us']
---------

['84047' '38567' '524027' '522490' '522432' '522506' '523456' '522375'
 '522196' '522294' '523748' '522288' '5228013' '522801']
---------

True


In [ ]:
from PIL import Image
from numpy import asarray
image = asarray(Image.open('../convertdemo/dataset/rand_6.jpg'))
image.shape

In [ ]:
image = Image.open('../convertdemo/dataset/rand_6.jpg')

In [ ]:
image.size

In [ ]:
from numpy import asarray